标题: 使用 # 来表示不同级别的标题，比如：

# 一级标题
## 二级标题
### 三级标题
粗体: 使用 ** 或 __ 来包围文本，例如：

**这是粗体文本** 或 __这是粗体文本__
斜体: 使用 * 或 _ 来包围文本，例如：

*这是斜体文本* 或 _这是斜体文本_
删除线: 使用 ~~ 来包围文本，例如：

~~这是带有删除线的文本~~
链接: 使用 []() 来创建一个链接，例如：

[百度](http://www.baidu.com)
图片: 使用 ![]() 来插入一个图片，例如：

![图片描述](图片链接)
无序列表: 使用 * 或 - 或 + 后接空格来创建无序列表，例如：

* 项目1
- 项目2
+ 项目3
有序列表: 使用数字和点后接空格来创建有序列表，例如：

1. 项目1
2. 项目2
3. 项目3
引用: 使用 > 来创建引用，例如：

> 这是一段引用
代码: 使用 ` ` 来插入内联代码，例如 这是一段代码。使用 ``` ``` 来插入代码块，

In [1]:
import os
os.environ['HTTP_PROXY'] = "127.0.0.1:10809"
os.environ['HTTPS_PROXY']="127.0.0.1:10809"
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0.1)

## 从key文件读取key值
* 输入参数: file_path 文件路径
* 返回值: 存有key的列表

In [3]:
def read_keys_from_file(file_path):
    key_pool = []
    with open(file_path, 'r') as file:
        for line in file:
            key = line.strip()  # 去掉每一行的换行符
            key_pool.append(key)  # 将每一行存入key池中
    return key_pool

In [4]:
# 读取key
file_path = "key.txt"  # 将这个替换为你的txt文件的路径
key_pool = read_keys_from_file(file_path)

## 读取PDF文档函数
* 参数:文件路径file_path,提取前n页
* 返回值:一个列表

In [5]:
import pdfplumber

def read_pdf(file_path, n):
    inner = []
    text = ""
    with pdfplumber.open(file_path) as pdf:
            # 提取前n页
            for i, page in enumerate(pdf.pages):
                # 如果已经处理了n页，就停止
                if i == n:
                    break
                inner.append(page.extract_text())
    return inner

# 按页数进行划分
* 参数:inner->一个装有每一页pdf的列表
* 参数:group_size->每个组的大小
* 返回值:一个列表

In [6]:
def group_inner(inner, group_size):
    # 将inner按照每n页为一组进行分割
    grouped_inner = [inner[i:i + group_size] for i in range(0, len(inner), group_size)]
    return grouped_inner

In [7]:
content = read_pdf(r'./data/KT820说明书V2.0 201909.pdf', 15)

In [8]:
inner = group_inner(content, 5)

In [9]:
print(len(inner))

3


In [10]:
from langchain.prompts import  ChatPromptTemplate
prompts = """
    ----
    {text}
    ----
    帮我把上面内容总结成2-4个目录，目录不要子目录，目录为简洁摘要，其它内容不用给我
    """

prompts2 = """
    abstract:
    ----
    {abstract}
    ----
    text:
    ----
    {text}
    ----

    跟据上面的摘要，找出下面内容中每个摘要对应的开始位置内容：
    返回格式:
    {respond_struction}

"""

prompts3 = """
    text:
    ----
    {text}
    ----

    Json
    '''
    {Json}
    '''
    根据上面的json的内容,从text中找到对应的段落并且返回符合逻辑的内容回来,text用---分割,json用'''来分割

    返回格式:
    {respond_struction}

"""
promptsTemplate = ChatPromptTemplate.from_template(prompts)
promptsTemplate2 = ChatPromptTemplate.from_template(prompts2)
promptsTemplate3 = ChatPromptTemplate.from_template(prompts3)

## 指定返回值格式
> 返回值格式采用json数据,包含三部分:关键词,开始词,结束词
### langchain的json格式
1. 先定义单个模式,Schema = ResponseSchema(name="",description="")
2. 然后将多个模式组合在一起response_schemas = [abstract_schema, start_schema, end_schema]
3. 通过结构输出解析函数解析成结构对象 = StructuredOutputParser(response_schemas = response_schemas)
4. 最后将结构对象解析成json格式的内容


In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
abstract_schema = ResponseSchema(name="abstract",description="这是摘要部分的内容, 也就是用来记录摘要的")
content_schema = ResponseSchema(name="content",description="返回和上面摘要部分相关的全部内容,有n个摘要就返回n个内容")

response_schemas = [abstract_schema, content_schema]
out_parse = StructuredOutputParser(response_schemas = response_schemas)
format_instructions = out_parse.get_format_instructions()

## 制作两条链来定位
* 第一条链用来摘要总结文本内容,第一条链用3.5的模型
* 第二条链用来根据摘要来定位文本内容,第二条链用3.5模型

In [12]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promptsTemplate)
# llm2 = ChatOpenAI(model_name='gpt-4-0613', temperature=0.2)
chain2 = LLMChain(llm=llm, prompt=promptsTemplate2)

In [ ]:
import random
abstract=[]
result = []
for text in inner:
    os.environ["OPENAI_API_KEY"] = key_pool[random.randint(0,len(key_pool)-1)]
    print(os.environ.get("OPENAI_API_KEY"))
    respones=chain.run(text = text)
    res = chain2.run(text=text,abstract=respones,respond_struction=format_instructions)
    abstract.append(respones)
    result.append(res)

In [ ]:
print(result[2])

In [ ]:
print(result)
print(len(result))
print(len(inner[2]))

1. [{},{},{}]
2. 输出文件